In [1]:
#### Projeto: Desafio ZAP
#### Programa para Treinar o Modelo Boosting com diferentes hiperparâmetros
#### Autor: Rodolfo Bugarin

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
#
# Carregar os Dataframes
#

In [4]:
X = pd.read_pickle('df_X_features.pickle')
y = pd.read_pickle('df_y_label.pickle') 

In [5]:
X_train = pd.read_pickle('df_X_train_features.pickle')
X_test = pd.read_pickle('df_X_test_features.pickle') 

In [6]:
y_train = pd.read_pickle('df_y_train_label.pickle')
y_test = pd.read_pickle('df_y_test_label.pickle') 

In [7]:
# Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit the training data
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
# Apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Boosting -  Fitting the best model

In [11]:
from sklearn import ensemble

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
tuned_parameters = {'n_estimators': [50, 150, 200], 'max_depth' : [50], 'min_samples_split': [50],
          'learning_rate': [0.01], 'loss': ['lad']}

In [13]:
clf = GridSearchCV(ensemble.GradientBoostingRegressor(), tuned_parameters, cv=3, scoring='r2')

In [14]:
%%time

clf.fit(X_train, y_train)

Wall time: 20min 17s


GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 150, 200], 'max_depth': [50], 'min_samples_split': [50], 'learning_rate': [0.01], 'loss': ['lad']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [15]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()

Best parameters set found on development set:

{'learning_rate': 0.01, 'loss': 'lad', 'max_depth': 50, 'min_samples_split': 50, 'n_estimators': 200}

Grid scores on development set:

0.230 (+/-0.064) for {'learning_rate': 0.01, 'loss': 'lad', 'max_depth': 50, 'min_samples_split': 50, 'n_estimators': 50}
0.581 (+/-0.160) for {'learning_rate': 0.01, 'loss': 'lad', 'max_depth': 50, 'min_samples_split': 50, 'n_estimators': 150}
0.666 (+/-0.177) for {'learning_rate': 0.01, 'loss': 'lad', 'max_depth': 50, 'min_samples_split': 50, 'n_estimators': 200}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.



In [16]:
predictions = clf.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE: %.4f" % mse)
r2 = r2_score(y_test, predictions)
print("R2: %.4f" % r2)
print()

MSE: 246843007502.5304
R2: 0.7563



In [17]:
#
# Salvar o modelo
#

In [18]:
modelo_clf = clf.get_params()
outfile = open('modelo_clf.pickle','wb')
pickle.dump(modelo_clf, outfile)
outfile.close()